In [2]:
# Analizar los resultados de los ejercicos 5.1. y 5.2.
# a) Por separado, comprobar si existe colinealidad entre las variables predictoras.
# b) Comparar los modelos de los ejercicios 5.1. y 5.2. entre sí y con los modelos de regresión simple del tema 4.

In [28]:
# Apartado a)
# Se comienza repitiendo el comienzo del ejercicio 5.1., en concreto se extraen del dataset la variable respuesta y la variable categórica, quedando
# únicamente las variables numéricas, sobre las cuales se lleva a cabo el proceso de imputación (pero no de estandarización)
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor # Para analizar posible colinealidad

housing = pd.read_csv('datasets/housing/housing.csv')
respuesta = housing['median_house_value']
housing = housing.drop('median_house_value', axis=1) # Se elimina del dataframe la columna (axis=1) correspondiente a la variable respuesta

# Para que la imputación no de error se elimina la variable categórica (no numérica)
housing_num = housing.drop('ocean_proximity', axis=1)

# Imputación
imputador = SimpleImputer(strategy='median')
imputador.fit(housing_num)
housing_num_imp = imputador.transform(housing_num) # El método transform devuelve un array de arrays en lugar de un dataframe
nombre_columnas = housing_num.columns
dic = {nombre_columnas[i]:housing_num_imp.transpose()[i] for i in range(len(nombre_columnas))} # Diccionario auxiliar 
housing_imp = pd.DataFrame(dic) # Se construye el dataset imputado a partir del diccionario auxiliar

print(housing_imp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20640 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
dtypes: float64(8)
memory usage: 1.3 MB
None


In [32]:
# Ahora ya se puede proceder con el análisis de colinealidad entre variables predictoras
ej1_VIF = pd.DataFrame()
ej1_VIF['variable'] = housing_imp.columns # Se crea una primera columna del dataframe con el nombre de las variables numéricas

# Se crea una segunda columna del dataframe con los resultados del VIF para cada variable
ej1_VIF['VIF'] = [variance_inflation_factor(housing_imp.values, i) for i in range(len(housing_imp.columns))]

ej1_VIF

,variable,VIF
0,longitude,614.390295
1,latitude,548.954416
2,housing_median_age,7.281102
3,total_rooms,29.032423
4,total_bedrooms,70.525566
5,population,16.173089
6,households,73.547059
7,median_income,8.003945


In [44]:
# La longitud y la latitud presentan valores del VIF muy elevados, lo cual sugiere un problema de colinealidad.
# Se repite el análisis con el modelo del ejercicio 5.2., que también tiene en cuenta la variable categórica. Se comienza repitiendo el proceso 
# seguido en el ejercicio 5.2.

from sklearn.preprocessing import OneHotEncoder # Para crear variable indicadora

# Es necesario definir una variable indicadora para la variable categórica 'ocean_proximity', a la que se llamará 'ocean_cat'.
ocean_cat = housing[['ocean_proximity']]

header_ocean_cat = housing['ocean_proximity'].unique() # Saca todos los posibles valores de la variable categórica

indicador_var = OneHotEncoder(sparse_output=False) 
# Se crea una matriz con cinco columnas y 20640 filas. Cada columna corresponde a uno de los cinco posibles valores para 'ocean_proximity'. Se asigna
# un 1 al valor que toma y un cero al resto de posibles valores.
ocean_indicadora = indicador_var.fit_transform(ocean_cat) 

ocean_df = pd.DataFrame(ocean_indicadora, columns=header_ocean_cat, index=housing.index) # Se guarda la matriz en un dataframe

housing_t = pd.merge(housing_imp, ocean_df, left_index=True, right_index=True)
housing_t

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,NEAR BAY,<1H OCEAN,INLAND,NEAR OCEAN,ISLAND
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,0.0,0.0,0.0,1.0,0.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,0.0,0.0,0.0,1.0,0.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,0.0,0.0,0.0,1.0,0.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,0.0,0.0,0.0,1.0,0.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,0.0,1.0,0.0,0.0,0.0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,0.0,1.0,0.0,0.0,0.0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,0.0,1.0,0.0,0.0,0.0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,0.0,1.0,0.0,0.0,0.0


In [48]:
# Se elimina la variable categórica <1H OCEAN, pues es la que se incluyó en el intercepto en el ejercicio 5.2.
housing_t = housing_t.drop('<1H OCEAN', axis=1)
print(housing_t.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20640 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   NEAR BAY            20640 non-null  float64
 9   INLAND              20640 non-null  float64
 10  NEAR OCEAN          20640 non-null  float64
 11  ISLAND              20640 non-null  float64
dtypes: float64(12)
memory usage: 1.9 MB
None


In [50]:
# Ahora ya se puede proceder con el análisis de colinealidad entre variables predictoras
ej2_VIF = pd.DataFrame()
ej2_VIF['variable'] = housing_t.columns
ej2_VIF['VIF'] = [variance_inflation_factor(housing_t.values, i) for i in range(len(housing_t.columns))]
ej2_VIF

,variable,VIF
0,longitude,942.605262
1,latitude,844.284917
2,housing_median_age,8.116462
3,total_rooms,30.259091
4,total_bedrooms,70.550282
5,population,16.381178
6,households,75.251325
7,median_income,8.822349
8,NEAR BAY,3.881851
9,INLAND,1.003045


In [52]:
# De nuevo, los valores VIF de la longitud y la latitud son muy elevados

In [59]:
# Apartado b)
# Comparamos los modelos a partir del coeficiente de determinación ajustado
# En el ejercicio 4.1. se concluyó que el mejor modelo de regresión simple se obtenía considerando como variable predictora 'median_income'. Se obtuvo
# un coeficiente R^2 = 0.473.
# En el ejercicio 5.1. se consideró un modelo de regresión múltiple con solo las variables numéricas. Se obtuvo R^2_adj = 0.635.
# En el ejercicio 5.2. se consideró un modelo de regresión múltiple con todas las variables. Se obtuvo R^2_adj = 0.645.
# Entonces, de acuerdo con el criterio del coeficiente de determinación, el mejor modelo es el de regresión lineal múltiple con todas las variables,
# aunque como se ha visto en el apartado a) presenta problemas de colinealidad.